<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


In [1]:
%%javascript
//useful TOC utils for ipy-notebooks! much thanks to the authors
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')


<IPython.core.display.Javascript object>

Creating a directory for datasets (the sources and those that will be created later on)

In [2]:
%%bash
[ -d datasets ] && echo "data directory already existing" || { echo "creating data directory \"./datasets\"" ; mkdir datasets; }

data directory already existing


Importing the needed packages

In [3]:
import os                  #some check operations
import time
import pandas as pd        #panda dataframes
import numpy as np         #numpy structures
import re                  #regex utils
import ipython_memory_usage.ipython_memory_usage as imu #memory usage checker

In [4]:
# imu.start_watching_memory() 

Downloading the source datasets from the [github repository]( https://github.com/gdv/foundationsCS-2018/tree/master/ex-data/project).

Thanks to a nice Pandas option, we don't need any url request (otherwise, for a better url handling we could have imported the `url` and the `request` modules).

However, since something might go wrong while downloading (i.e. connection issues), we better use the defensive try/except prassi.

If the datasets have been already downloaded and saved, the offline one will be used.

In [5]:
#!rm datasets/* #just for checking/initializing

if not os.path.isfile("datasets/googleplaystore.csv"):
    print ('downloading "googleplaystore.csv" from source')
    try: #something may go wrong
        url = 'https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore.csv'
        gps = pd.read_csv(url,index_col=0,parse_dates=[0]) #GooglePlayStore dataframe
    except Exception as e:
        print(e)
        #exit
    
    gps.to_csv("./datasets/googleplaystore.csv")
    
else:
    print('"googleplaystore.csv" found')
    gps = pd.read_csv("./datasets/googleplaystore.csv")

gps.head(2)



"googleplaystore.csv" found


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [6]:
#!rm ./datasets/googleplaystore_user_reviews.csv
if not os.path.isfile("datasets/googleplaystore_user_reviews.csv"):
    
    print ('downloading "googleplaystore_user_reviews.csv" from source')
    
    try: #something may go wrong
        url = 'https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv'
        gpsr = pd.read_csv(url,index_col=0,parse_dates=[0]) #GooglePlayStoreReviews dataframe
    except Exception as e:
        print(e)
        
    gpsr.to_csv("./datasets/googleplaystore_user_reviews.csv")

else:
    
    print('"googleplaystore_user_reviews.csv" found')
    gpsr = pd.read_csv("./datasets/googleplaystore_user_reviews.csv")
    
    

gpsr.head(2)

"googleplaystore_user_reviews.csv" found


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462




In exercise 2, I recognized an issues concerning row n.10472, since the _Category_ field is not shown and everything is shifted to the left.



In [7]:
gps.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


I decided to remove this line, having no knowledge of the domain to fullfill the missing structure with any plausible value.

In [8]:
gps=gps.drop(10472)

start_time=time.time()

# <span style="color:red"> Exercise 1 </span>

#### Convert the app sizes to a number

Let us use a Regex for size pattern recognition

In [9]:


num_pattern = re.compile('(?P<num>\d+\.{0,1}\d*)(?P<prefix>\w*)')



Then define a function returning the corresponding numeric value for the International System Units

In [10]:

def SI2numeric(unit):
    if unit == 'G': return 1000000000
    if unit == 'M': return 1000000
    if unit == 'k': return 1000
    return 1    


Eventually define a function executing the proper conversion

In [11]:

def Size2Byte(Size):
    searched = num_pattern.search(Size)
    if searched:
        prefix = searched.group('prefix')
        unit = SI2numeric(prefix)
        num = float(searched.group('num'))
        return int(num*unit)
    else:
        return np.nan
    

Applying on the dataframe

In [12]:
%%time
gps['SizeInBytes'] = gps['Size'].apply(Size2Byte)


CPU times: user 13.5 ms, sys: 865 µs, total: 14.3 ms
Wall time: 14.3 ms


In [13]:
gps[['App','Size','SizeInBytes']].tail(2)

,App,Size,SizeInBytes
10839,The SCP Foundation DB fr nn5n,Varies with device,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,19M,19000000.0


# <span style="color:red"> Exercise 2 </span>
#### Convert the number of installs to a number

First of all, let us check the situation

In [14]:
gps.groupby('Installs')['Installs'].size().head()

Installs
0                1
0+              14
1+              67
1,000+         907
1,000,000+    1579
Name: Installs, dtype: int64

Quick regex to strip characters

In [15]:
print ( "initial data type is", gps['Installs'].dtypes)

gps['NumInstalls'] = [re.sub('\\D', '', entry) for entry in gps['Installs']]

initial data type is object


Convert the data type

In [16]:
gps['NumInstalls'] = gps['NumInstalls'].apply(int)

print ( "data type is now", gps['NumInstalls'].dtypes)

gps[['App','Installs','NumInstalls']].head(3)

data type is now int64


,App,Installs,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000


# <span style="color:red"> Exercise 3 </span>
#### Transform “Varies with device” into a missing value

In [17]:
gps.replace('Varies with device', np.nan,inplace=True)

In [18]:
gps.tail(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,NumInstalls
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",NaN,NaN,NaN,1000
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",NaN,NaN,19000000.0,10000000


# <span style="color:red"> Exercise 4 </span>
#### Convert Current Ver and Android Ver into dotted number (e.g. 4.0.3 or 4.2)

A mandatory inspection over _Current Ver_

In [19]:
gps['Current Ver'].value_counts().tail()

3.3.99      1
3.15317     1
1.0.109     1
1009001     1
7.12.0.1    1
Name: Current Ver, dtype: int64

Regex pattern to identify numbers (and non-numeric values) in the format _num.num.num_

In [20]:
CurrVer = re.compile('(?P<ver>[\d+\.]*\d)') 

def DotCurrVer(version):
    #handling missing values
    if pd.isnull(version):
        return np.nan
    searched=CurrVer.search(version)
    if searched is None:
        return np.nan
    else:
        return searched.group('ver')

In [21]:
gps['DotCurrVer']=gps['Current Ver'].apply(DotCurrVer)


Apparently no great differences...

In [22]:
gps[['App','Current Ver','DotCurrVer']].head(3)

,App,Current Ver,DotCurrVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,1.0.0,1.0.0
1,Coloring book moana,2.0.0,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",1.2.4,1.2.4


But... let us highlight some changes

In [23]:
%%time
gps[gps['Current Ver'].apply(lambda x: type(x)==str)&gps['DotCurrVer'].apply(lambda x: type(x)==float)][['App','Current Ver','DotCurrVer']].head()

CPU times: user 7.07 ms, sys: 0 ns, total: 7.07 ms
Wall time: 6.68 ms


,App,Current Ver,DotCurrVer
1423,Self Healing,Public.Heal,NaN
2501,HHS,Initial,NaN
3611,Latest Emmanuella Comedy Video,Natalia Studio Development,NaN
5218,AI Image Recognizer (beta),closed,NaN
5289,Ak-47 Wallpapers 2018,newversion,NaN


A mandatory inspection over _Android Ver_

In [24]:
gps['Android Ver'].value_counts().tail()

5.0 - 6.0      1
7.0 - 7.1.1    1
5.0 - 7.1.1    1
4.1 - 7.1.1    1
2.2 - 7.1.1    1
Name: Android Ver, dtype: int64

Regex pattern to identify numbers (and non-numeric values) in the format _num.num.num_

In [25]:
First_v=re.compile('^(?P<First>\d+\.\d+\.{0,1}\d*)') #first supported version
Last_v=re.compile('(?P<Last>\d+\.\d+\.{0,1}\d*)$')   #last supported version


def FirstSuppVer(version):
    #handling missing data
    if pd.isnull(version):
        return np.nan
    searched=First_v.search(version)
    if searched is  None:
        return np.nan
    else:
        return searched.group('First')
    
def LastSuppVer(version):
    if pd.isnull(version):
        return np.nan
    searched=Last_v.search(version)
    if searched is None:
        return 9
    else:
        return searched.group('Last')

In [26]:
gps['FirstSuppVer']=gps['Android Ver'].apply(FirstSuppVer)
gps['LastSuppVer']=gps['Android Ver'].apply(LastSuppVer)
gps[['App','Android Ver','FirstSuppVer','LastSuppVer']].tail()

,App,Android Ver,FirstSuppVer,LastSuppVer
10836,Sya9a Maroc - FR,4.1 and up,4.1,9
10837,Fr. Mike Schmitz Audio Teachings,4.1 and up,4.1,9
10838,Parkinson Exercices FR,2.2 and up,2.2,9
10839,The SCP Foundation DB fr nn5n,NaN,NaN,NaN
10840,iHoroscope - 2018 Daily Horoscope & Astrology,NaN,NaN,NaN


# <span style="color:red"> Exercise 5 </span>
#### Remove the duplicates

First of all, let us identify the entirely duplicated rows:

In [27]:
print('Duplicated rows: ', gps[gps.duplicated()].shape[0])

Duplicated rows:  483


Then, we check the duplicated apps:

In [28]:
print('Duplicated Apps: ', gps[gps.duplicated('App')].shape[0])

Duplicated Apps:  1181


We then choose to keep, for each duplicated app, only the row with the highest number of reviews.

In order to do that, we sort the database according to the _Reviews_ field, dropping all but the first entry.

In [29]:
gps.Reviews = gps.Reviews.apply(pd.to_numeric); gps.Reviews.dtype

dtype('int64')

In [30]:
gps=gps.sort_values('Reviews',ascending=False).drop_duplicates('App',keep='first').reset_index(drop=True)

Brief check:

In [31]:
print('Duplicated Apps: ',gps[gps.duplicated('App')].shape[0])
gps.head(2)

Duplicated Apps:  0


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,SizeInBytes,NumInstalls,DotCurrVer,FirstSuppVer,LastSuppVer
0,Facebook,SOCIAL,4.1,78158306,NaN,"1,000,000,000+",Free,0,Teen,Social,"August 3, 2018",NaN,NaN,NaN,1000000000,NaN,NaN,NaN
1,WhatsApp Messenger,COMMUNICATION,4.4,69119316,NaN,"1,000,000,000+",Free,0,Everyone,Communication,"August 3, 2018",NaN,NaN,NaN,1000000000,NaN,NaN,NaN


# <span style="color:red"> Exercise 6 </span>
#### For each category, compute the number of apps

Having already removed duplicated lines, we can easily proceed this way:

In [32]:
 gps['Category'].value_counts().sort_values().head(3)

BEAUTY       53
COMICS       56
PARENTING    60
Name: Category, dtype: int64

# <span style="color:red"> Exercise 7 </span>
#### For each category, compute the average rating

In [33]:
gps.groupby('Category')[['Rating']].mean().sort_values(by = 'Rating', ascending = False).head(3)

,Rating
Category,
EVENTS,4.435556
ART_AND_DESIGN,4.359322
EDUCATION,4.354717


# <span style="color:red"> Exercise 8 </span>
#### Create two dataframes: one for the genres and one bridging apps and genders. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

I noticed that some Genres entry are a bit strange: sometimes there are two identical genres..

In [34]:
gps['Genres'].loc[9037]

'Education;Education'

But sometimes those are distincts. 

In [35]:
 #seems like some genre assignment is not clear.
gps['Genres'].loc[9051]

'Role Playing;Education'

In order not to discard some data, we decide then to keep both the genres in the new dataframe, using a column for the "Main_genre" and another for a "Sec_genre"

In [36]:
gps['SplitGenres'] = [g.split(';') for g in gps['Genres']]
genres = gps.SplitGenres.apply(pd.Series) #genres dataframe
genres = genres.rename(columns = {0:'Main_genre', 1:'Sec_genre'})

So that now it makes a bit of sense:

In [37]:
genres.loc[[9051]]

,Main_genre,Sec_genre
9051,Role Playing,Education


Let us check how many good data we can optionally "save" this way: (those are a bit few)

In [38]:

genres[(genres.Sec_genre!=genres.Main_genre)& ~genres.Sec_genre.isnull()].count()

#genres[(genres.Sec_genre!=genres.Main_genre)& ~genres.Sec_genre.isnull()] #for eventual display


Main_genre    356
Sec_genre     356
dtype: int64

Let us create a local copy

In [39]:
genres.to_csv("./datasets/Genre_Table.csv")

Then let us consider the bridging table: (we decide to discard the "Sec\_genre" approach, since the entries were few if compared to the whole amount)

In [40]:
apps = gps['App']
#Bridging table
bridgintable = pd.merge(genres, apps.to_frame(), left_index = True, right_index = True).melt(id_vars = ['App'], value_name = "Main_genre").drop("variable", axis = 1).dropna() #we decide to drop na values
bridgintable.head(3)

,App,Main_genre
0,Facebook,Social
1,WhatsApp Messenger,Communication
2,Instagram,Social


A brief check:

In [41]:
bridgintable[bridgintable['App'] == 'Pixel Draw - Number Art Coloring Book']

,App,Main_genre
4831,Pixel Draw - Number Art Coloring Book,Art & Design
14490,Pixel Draw - Number Art Coloring Book,Creativity


Some csv export

In [42]:
bridgintable.to_csv("./datasets/Bridging_table.csv")

In [43]:
gps.to_csv("./datasets/Processed_GooglePlayStore.csv")

# <span style="color:red"> Exercise 9 </span>
#### For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [44]:
def MakeTrue(row_entry, genre):
    row_entry[genre]=True
       
def WhatGenre(genres_col, row_entry):
    genre_list=gps.Genres
    genre_lst=genres_col.split(';')
    [ MakeTrue(row_entry, genre) for genre in genre_lst ] 
    return row_entry


<a id='another_cell'></a>

In [45]:
%%time
gps=gps.apply(lambda x: WhatGenre(x['Genres'], x), axis = 1)


CPU times: user 8.72 s, sys: 93.7 ms, total: 8.81 s
Wall time: 8.63 s


In [46]:
#%%time
#just for correctness, let us put the other values as (False) Bool as well

for elem in gps.Genres:
    elem=elem.split(';')
    for e in elem:
        gps[e].fillna(value=False, inplace=True)

In [47]:
#gps.tail(3)

gps[gps['Art & Design'].apply(lambda x: x==1)].head(2) #let's pick up some true values here and there

,Action,Action & Adventure,Adventure,Android Ver,App,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,...,SplitGenres,Sports,Strategy,Tools,Travel & Local,Trivia,Type,Video Players & Editors,Weather,Word
809,False,False,False,NaN,Textgram - write on photos,False,True,False,False,False,...,[Art & Design],False,False,False,False,False,Free,False,False,False
964,False,False,False,4.1 and up,ibis Paint X,False,True,False,False,False,...,[Art & Design],False,False,False,False,False,Free,False,False,False


# <span style="color:red"> Exercise 10 </span>
#### 10. For each genre, compute the average rating. What is the genre with highest average?

First of all, we check for any missing values, better to be aware of it.However, those will not represent a problem since the averaging function will by default ignore them.

In [48]:
gps['Rating'].isna().any()

True

We then merge the main table with the previouly created "bridgintable", so to obtain a "ratingtable" from where it is easy to calculate the average rating for each genre.

In [49]:
ratingtable = bridgintable.merge(gps, on = 'App')
ratingtable[['App','Genres','Main_genre']].head()

,App,Genres,Main_genre
0,Facebook,Social,Social
1,WhatsApp Messenger,Communication,Communication
2,Instagram,Social,Social
3,Messenger – Text and Video Chat for Free,Communication,Communication
4,Clash of Clans,Strategy,Strategy


To satisfy the "for each genre" request, we perform a groupby

In [50]:
ratingtable.groupby('Main_genre')[['Rating']].mean().sort_values(by = 'Rating', ascending = False).head(3)

,Rating
Main_genre,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358065


The genre with the highest average rating is returned through the idxmax() function

In [51]:
ratingtable.groupby('Main_genre')['Rating'].mean().idxmax()

'Events'

In [52]:
ratingtable.groupby('Main_genre')['Rating'].mean().max()

4.435555555555556

# <span style="color:red"> Exercise 11 </span>
#### 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

First of all, let us clean the price field, eliminating the extra characters. Thus, we obtain an integer-type field.

In [53]:
gps['IntPrice'] = [re.sub('\\D','', price) for price in gps['Price']]
gps['IntPrice'] = gps['IntPrice'].apply(int)

We then calculate the approximate income, following the aforementioned suggestion

In [54]:
gps['Income'] = gps['IntPrice'] * gps['NumInstalls']
gps[['App', 'Income']].sort_values(by = 'Income', ascending = False).head(3)

,App,Income
172,Minecraft,6990000000
3986,I am rich,3999900000
4441,I Am Rich Premium,1999950000


# <span style="color:red"> Exercise 12 </span>
#### 12. For each app, compute its minimum and maximum Sentiment_polarity

A brief review of the review table

In [55]:
gpsr.head(1)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.0,0.533333


As always, we check for missing value. Those are found again. However, as in the previous case, it is not going to be a problem, since we will use functions which will basically ignore those NaN values.

In [56]:
gpsr['Sentiment_Polarity'].isna().any() 

True

We compute the Min and Max Sentiment polarity for each app, exploiting a groupby on apps 

In [57]:
min_sent = pd.DataFrame(gpsr.groupby('App')['Sentiment_Polarity'].min())

In [58]:
max_sent = pd.DataFrame(gpsr.groupby('App')['Sentiment_Polarity'].max())

Merging the two df, we obtain the desired table

In [59]:
min_sent.merge(max_sent, on='App',suffixes=['_min','_max']).head()

,Sentiment_Polarity_min,Sentiment_Polarity_max
App,,
10 Best Foods for You,-0.8000,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.1125,0.910000
11st,-1.0000,1.000000
1800 Contacts - Lens Store,-0.3000,0.838542
1LINE – One Line with One Touch,-0.8250,1.000000


# <div class="alert alert-block alert-warning">Remark: Total Execution Time</div>

In [60]:
print("Program executed in %s seconds" % (time.time()-start_time))

Program executed in 11.84950852394104 seconds


<b> The "bottleneck" of the entire notebook is [this operation](#another_cell). <b>

One way to improve performance on this passage would have been writing a lambda one-liner, avoiding two function calls for every row. By testing this solution i obtained a %time performance of ~4 sec instead of ~8sec (meaning a ~8 sec of total execution time in the end). 

However, for clarity sake i've kept the function definition, much more "linear".